<a href="https://colab.research.google.com/github/whdigrace/kfq_ML_2023/blob/main/%EB%AC%B8%EC%84%9C%EC%9C%A0%EC%82%AC%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
def cos_similarity(v1,v2):
  dp = np.dot(v1,v2)
  vs1 = np.sqrt(sum(np.square(v1)))
  vs2 = np.sqrt(sum(np.square(v2)))
  return dp/(vs1*vs2)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
doc_list = [
    'if you take the blue pill, the story ends',
    'if you take the red pill, you stay in Wonderland',
    'if you tkae the red pill, I show you how deep the rabbit hole goes'
]

In [5]:
tv = TfidfVectorizer()
fvs = tv.fit_transform(doc_list)
fvs.shape

(3, 18)

In [6]:
fvs

<3x18 sparse matrix of type '<class 'numpy.float64'>'
	with 28 stored elements in Compressed Sparse Row format>

In [7]:
fvd = fvs.todense()
fvd[0]

matrix([[0.4075631 , 0.        , 0.4075631 , 0.        , 0.        ,
         0.24071333, 0.        , 0.24071333, 0.        , 0.        ,
         0.        , 0.        , 0.4075631 , 0.30996224, 0.48142667,
         0.        , 0.        , 0.24071333]])

In [8]:
vect1 = np.array(fvd[0]).reshape(-1)
vect1

array([0.4075631 , 0.        , 0.4075631 , 0.        , 0.        ,
       0.24071333, 0.        , 0.24071333, 0.        , 0.        ,
       0.        , 0.        , 0.4075631 , 0.30996224, 0.48142667,
       0.        , 0.        , 0.24071333])

In [10]:
vect2 = np.array(fvd[1]).reshape(-1)

In [11]:
score = cos_similarity(vect1,vect2)
print(f"유사도:{score:3f}")

유사도:0.423840


In [12]:
doc_list = [
    'if you take the blue pill, the story ends',
    'if you take the blue pill, the story starts',
    'if you tkae the red pill, I show you how deep the rabbit hole goes'
]

In [13]:
tv = TfidfVectorizer()
fvs = tv.fit_transform(doc_list)
fvs.shape

fvd = fvs.todense()

In [15]:
vect1 = np.array(fvd[0]).reshape(-1)  #1차원 줄이기
vect2 = np.array(fvd[1]).reshape(-1)

score = cos_similarity(vect1,vect2)
print(f"유사도:{score:3f}")

유사도:0.806838


In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
cosine_similarity(fvs[0],fvs)

array([[1.        , 0.80683757, 0.36185494]])

In [18]:
d_path = '/content/drive/MyDrive/topics'

In [19]:
import pandas as pd
import glob,os

In [20]:
fs = os.path.join(d_path,"*.data")
all_files = glob.glob(fs)
saf = sorted(all_files)
fnl = []
opt = []
for fn in saf:
  df = pd.read_table(fn,index_col=None,
                header=0,encoding='latin1')
  fn_ = fn.split('/')[-1]
  fn2 = fn_.split('.')[0]
  fnl.append(fn2)
  opt.append(df.to_string())
print(fnl)
print(opt)

['accuracy_garmin_nuvi_255W_gps', 'bathroom_bestwestern_hotel_sfo', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps', 'eyesight-issues_amazon_kindle', 'features_windows7', 'fonts_amazon_kindle', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007', 'keyboard_netbook_1005ha', 'location_bestwestern_hotel_sfo', 'location_holiday_inn_london', 'mileage_honda_accord_2008', 'navigation_amazon_kindle', 'parking_bestwestern_hotel_sfo', 'performance_honda_accord_2008', 'performance_netbook_1005ha', 'price_amazon_kindle', 'price_holiday_inn_london', 'quality_toyota_camry_2007', 'room_holiday_inn_london', 'rooms_bestwestern_hotel_sfo', 'rooms_swissotel_chicago', 'satellite_garmin_nuvi_255W_

In [22]:
from nltk.stem import WordNetLemmatizer
import nltk
import string
from sklearn.cluster import KMeans
nltk.download('all')
df = pd.DataFrame({'fn':fnl, 'opt':opt})
rpd = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(rpd)))
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
                             ngram_range=(1,2),
                             min_df=0.05,
                             max_df=0.85)
tf_data = tfidf_vect.fit_transform(df['opt'])

km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(tf_data)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_
df['cluster_label'] = cluster_label

from nltk.stem import WordNetLemmatizer
import nltk
import string
from sklearn.cluster import KMeans
nltk.download('all')
df = pd.DataFrame({'fn':fnl, 'opt':opt})
rpd = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(rpd)))
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
                             ngram_range=(1,2),
                             min_df=0.05,
                             max_df=0.85)
tf_data = tfidf_vect.fit_transform(df['opt'])

km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(tf_data)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_
df['cluster_label'] = cluster_label

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [23]:
df.columns

Index(['fn', 'opt', 'cluster_label'], dtype='object')

In [26]:
fil = df['cluster_label'] == 0
fil_index = df[fil].index
fil_index

Int64Index([ 0,  2,  3,  4,  5,  8,  9, 10, 11, 12, 19, 23, 26, 27, 33, 34, 35,
            36, 41, 42, 43, 44, 48, 49, 50],
           dtype='int64')

In [27]:
cdn = df[fil]['fn']
cdn

0       accuracy_garmin_nuvi_255W_gps
2          battery-life_amazon_kindle
3          battery-life_ipod_nano_8gb
4         battery-life_netbook_1005ha
5               buttons_amazon_kindle
8     directions_garmin_nuvi_255W_gps
9        display_garmin_nuvi_255W_gps
10      eyesight-issues_amazon_kindle
11                  features_windows7
12                fonts_amazon_kindle
19            keyboard_netbook_1005ha
23           navigation_amazon_kindle
26         performance_netbook_1005ha
27                price_amazon_kindle
33     satellite_garmin_nuvi_255W_gps
34        screen_garmin_nuvi_255W_gps
35               screen_ipod_nano_8gb
36              screen_netbook_1005ha
41           size_asus_netbook_1005ha
42                sound_ipod_nano_8gb
43         speed_garmin_nuvi_255W_gps
44                     speed_windows7
48       updates_garmin_nuvi_255W_gps
49                video_ipod_nano_8gb
50         voice_garmin_nuvi_255W_gps
Name: fn, dtype: object

In [28]:
fil_index[0]

0

In [29]:
fil = df['cluster_label'] ==1
fil_index = df[fil].index
src = tf_data[fil_index[0]]
comp = tf_data[fil_index[1:]]

In [30]:
np.mean(cosine_similarity(src,comp))

0.3132580298559421

In [31]:
fil = df['cluster_label'] ==2
fil_index = df[fil].index
comp = tf_data[fil_index]
np.mean(cosine_similarity(src,comp))

0.04594791373202093

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import